# Graph Creation Breda
This notebook generates a graph based on pre-collected OSM data. The input must be a GeoJSON file in which each row contains a geometry of type LineString, MultiLineString, Polygon, or MultiPolygon.

The required data can be obtained by running the notebooks get_data.ipynb, assign_risk.ipynb, and assign_no_fly_zones.ipynb, in that specific order.
The resulting GeoJSON file from those steps serves as input for this notebook; you only need to specify the city name.

Make sure you also identify the corresponding distribution center for the selected city, as this must be provided as well.

This notebook is an example for Breda.

In [ ]:
from model_get_data import read_geojson
from model_create_graph import create_graph_with_nodes
from model_add_linear_infrastructures import add_linesstrings_to_graph
from model_add_polygon_infrastructures import add_grid_polygons_to_graph
from model_connect_postnl_nodes import connect_to_nearest_edge
from model_delete_edges_in_no_fly import remove_no_fly_zones_from_graph
from model_check import fill_missing_edge_heights_from_csv, diagnose_graph

import pickle

### 0. Specifying the Area
Set the city variable to indicate which area the data was collected for.
Set the distribution variable to specify the corresponding distribution center.
If you're unsure which distribution center belongs to the selected city, you can look it up in the file:
model/postNL/output/postnl_distribution_cleaned.json.

In [ ]:
city = 'breda' # use lower case
depot = ['Breda'] # use capitalized

### 0. Get Data
Uses the read_geojson function to load and prepare the data for the selected area, so that a graph can be constructed from the OSM data.

In [ ]:
gdf, lines_gdf, polygons_gdf, post_nl_gdf, no_fly_zones_gdf = read_geojson(f'/Users/cmartens/Documents/thesis_cf_martens/3.no_fly_zones/output/data_for_graph_{city}.geojson', depot, all=True)

In [ ]:
no_fly_zones_gdf

## 1. Create Initial Graph with Nodes
Creates an unconnected graph by adding all PostNL points and the distribution center as nodes.

In [ ]:
G = create_graph_with_nodes(post_nl_gdf)

## 2. Add LineStrings to Graph
Adds all LineString and MultiLineString geometries to the graph as edges. Ensures they are connected to each other and to the existing nodes by snapping nearby points together.

In [ ]:
G_with_linestrings = add_linesstrings_to_graph(G, lines_gdf)

## 3. Add polygons to 

Generates a grid for each polygon and connects the grid points to the polygon boundaries. Adds all polygons to the graph and ensures they are properly snapped to the existing graph structure, so that all elements are correctly connected.

In [ ]:
G_with_grids_and_boundaries = add_grid_polygons_to_graph(G_with_linestrings, polygons_gdf, lines_gdf)

## 4. Connect postnl and distribution points to graph

All points are linked to the nearest location in the graph using a connector, provided the distance does not exceed max_connection_distance.

In [ ]:
G_with_grids_and_boundaries_connected = connect_to_nearest_edge(G_with_grids_and_boundaries, max_connection_distance=10)

## 5. Delete edges inside nofly zones

All edges located within no-fly zones are detected and removed from the graph.

In [ ]:
G_final = remove_no_fly_zones_from_graph(G_with_grids_and_boundaries_connected, no_fly_zones_gdf)

## 6. Save graph

Graph will be saved to output folder.

In [ ]:
with open(f"output/{city}_raw.pkl", "wb") as f:
    pickle.dump(G_final, f)

print(f"Graph saved to output/{city}_raw.pkl")

## 7. Check graph

Use diagnose_graph to check if all nodes and edges are added correctly. If some height are missed these can still be added by using fill_missing_edge_heights_from_gdf.

In [ ]:
diagnose_graph(G_final)

In [ ]:
G_final = fill_missing_edge_heights_from_csv(G_final)


In [ ]:
diagnose_graph(G_final)

## 8. Save graph

In [ ]:
with open(f"output/{city}.pkl", "wb") as f:
    pickle.dump(G_final, f)

print(f"Graph saved to output/{city}.pkl")